<a href="https://colab.research.google.com/github/Lewis34cs/corona_audio/blob/main/covid_proj_p1_xrays.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Project Part 1: Chest X-Ray Multi Classification

>Author: Chris Lewis

>Contact info: lewis34cs@gmail.com


>- Link to [Project repo](https://github.com/Lewis34cs/corona_audio)
  - Part 1: [`covid_proj_p1_xrays.ipynb`](https://colab.research.google.com/drive/175EDoT7ACYu5JbIgEoFx9DboHzGiWdq2#scrollTo=2DJlDc99-sC5)(This Notebook)
  - Part 2: [`covid_proj_p2_audio.ipynb`](https://colab.research.google.com/drive/1s4z-GJnNkE2NCx5MVccXxXzvBIc__tTX#scrollTo=WjwyDrj8S80W)

#### COVID-19 Exploration in statistics and diagnosis via imaging and audio

>Since the beginning of 2020, COVID-19 has run rampant throughout the entirety of the globe, resulting in over 100 million cases and 2 million deaths. While vaccines are beginning to be pushed out as our new line of defense, my project's goal is to try and identify other ways to detect COVID-19 in a patient to help slow the spread. In this project, I focus on two different ways to identify COVID-19: multi classification via chest x-rays and binary classification through coughing audio. Both parts will be using Sequential Convolutional Neural Networks. This notebook will focus on COVID-19 diagnosis using chest x-ray images obtained from Kaggle's API.

>To run the Google Colab code, you must allow Google Colab to access an account. I will provide links for the data below. Please be sure to save zipped folders for these audio datasets into your google drive so you can access them on Google Colab. The chest x-ray database was obtained via Kaggle's API.

>**Chest X-ray Database**: https://www.kaggle.com/tawsifurrahman/covid19-radiography-database
---
>**Link for functions and zipped audio datasets**: https://drive.google.com/drive/folders/1IQ758RksB0ayUhOohfGh8s25MT4taMsI?usp=sharing


Click on the "add to drive" button once you've clicked on the Google Drive link above to add the datasets and functions to your Google Drive
<img src='https://drive.google.com/uc?id=1YYdx9_xhHRsYLqV2X7yUJauG9zKX_LoX'>

#### Mounting Google Drive

>We must run this in order for colab to be able to access our google drive. We can have google colab mount our google drive by either running the cell below or clicking on the folder icon on the left navigation bar and then clicking on the "Mount Drive" button at the top of the navigation pane.

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# We need to import these libraries in order to access the shared folder
import os, sys

In [4]:
# We must install these libraries for our imported functions folder to work
!pip install split-folders
!pip install split-folders tqdm
!pip install pydub
!pip install soundfile
!pip install ffmpeg-python
!pip install zip_files
!pip install lime
!pip install tensorflow_addons

     |████████████████████████████████| 276kB 15.5MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp37-none-any.whl size=283846 sha256=a933ec2d81f29c63a790741e4cf78df05aafae1b8e8e0bdbc15cc5a2eb65ab4e
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
Successfully built lime
     |████████████████████████████████| 706kB 15.5MB/s 


>Set the 'FOLDER' variable as the filepath where the folder containing functions.py and audio datasets are located in your Google Drive.

In [5]:
FOLDER = '/content/drive/MyDrive/chris_lewis_capstone/'
os.listdir(FOLDER)

['public_dataset.zip',
 'virufy_data-main.zip',
 '__pycache__',
 'public_dataset_v2.zip',
 'functions.py']

>Defining a path variable (our FOLDER) to add a specific path for the interpreter to search for and then load the filepath into the extensions in order to import all the functions from the functions.py file.

In [6]:
sys.path.append(FOLDER)

In [7]:
%load_ext autoreload
%autoreload 2
from functions import *

#### Importing Libraries

>These are the libraries we used in the project. Note that it is not necessary to run these cells because the functions.py file has already imported the libraries for us.

In [8]:
#Setting seeds first in order to achieve better consistency in scoring
import numpy as np
import tensorflow as tf

np.random.seed(321)
tf.random.set_seed(321)

In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import os, sys, glob, shutil

In [10]:
from sklearn import metrics
from sklearn.dummy import DummyClassifier
from sklearn.utils import class_weight
from skimage.segmentation import mark_boundaries
import time
from IPython.display import Image, display
from PIL import Image as Im
import cv2
import shutil
from google.colab import files

# tensorflow libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import models, layers, optimizers, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [11]:
import splitfolders
import lime
from lime import lime_image
from zipfile import ZipFile

In [12]:
# setting plot background style
plt.style.use('dark_background')

# PART 1: Classifying COVID-19 Chest X-ray Images via Sequential Convolutional Neural Network

In [13]:
# Viewing where we are in the directory
os.path.abspath(os.curdir)

'/content'

## Obtaining COVID CXR dataset via Kaggle API

>In order to obtain the chest x-ray database from Kaggle, we make a call to Kaggle's API using an API key. This database still updates regularly, which means that more images are placed into each class, giving us more data to train off of. Instead of downloading the file manually to our google drive, we prefer this route in order to avoid having to constantly check for database updates. 

>The dataset for this part can be found on Kaggle:
-  Chest X-ray database: 
  - https://www.kaggle.com/tawsifurrahman/covid19-radiography-database. 
- For instructions on how to get an API key: https://www.kaggle.com/docs/api

>Once you have the API key, please upload it in the cell below.
The below cells are retrieved from Kaggle.

**UPDATE** On March 6th 2021, the Kaggle dataset used in this project was updated - this included more images and a new class: lung_opacity. As of right now, this new class has not yet been included in the majority of the below code. 

In [14]:
# Hiding my key by having a variable store the info
API_key = files.upload()

Saving kaggle.json to kaggle.json


In [15]:
# Creating a hidden folder then copying my API key and putting it in that file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [16]:
# Permissions of 600 mean that I (the owner) have full read and write access to the 
# file, while no other user can access or edit the file
!chmod 600 ~/.kaggle/kaggle.json

In [17]:
# Downloading the zipped dataset via Kaggle's API
!kaggle datasets download -d tawsifurrahman/covid19-radiography-database

 97% 721M/745M [00:14<00:00, 55.2MB/s]
100% 745M/745M [00:14<00:00, 53.6MB/s]


#### Unzipping Folder Contents

>Unzipping the file and extracting to the root directory. If you chose to directly download the database from the kaggle page instead of making a call to the API, replace the 'f_name' variable with the location of the zipped file in your Google Drive. Do not unzip the zipped folder before loading into Google Colab.

In [18]:
f_name = "covid19-radiography-database.zip"

with ZipFile(f_name, 'r') as zipf:
      zipf.extractall()
      print('Done')

Done


In [19]:
# removing unnecessary files
!rm /content/covid19-radiography-database.zip
!rm /content/kaggle.json

In [20]:
# Setting directory location to root directory
%cd ~
%cd ..

/root
/


In [23]:
# setting a variable called cxr_base_folder that contains the directory of where 
# our chest x-ray images are
cxr_base_folder = ensure_filepath('/content/COVID-19_Radiography_Dataset')

In [24]:
# Viewing the contents within our cxr_base_folder
folder_contents(cxr_base_folder)

Contents of /content/COVID-19_Radiography_Dataset/:



['COVID',
 'COVID.metadata.xlsx',
 'Lung_Opacity',
 'Lung_Opacity.metadata.xlsx',
 'Normal',
 'Normal.metadata.xlsx',
 'README.md.txt',
 'Viral Pneumonia',
 'Viral Pneumonia.metadata.xlsx']

#### Identifying the number of images in our dataset

In [32]:
total_image_number = num_items(cxr_base_folder, 
                               targets=['COVID', 'Normal', 'Viral Pneumonia', 'Lung_Opacity'])

/content/COVID-19_Radiography_Dataset/
Number of items in COVID: 3616
Number of items in Normal: 10192
Number of items in Viral Pneumonia: 1345
Number of items in Lung_Opacity: 6012
Total number of items: 21165


>From here, we view the extracted contents and count the number of images per class along with the total image count. We see that we are working with 3800+ images.

### Splitting our images into train, test, and validation sets

>Here, we are making a folder to store our train, test, and validation folders. If you would like to store the folders in another filepath, please specify where you would like to store your train, test, and validation folders in the 'cxr_folders' variable. Note that this folder is temporary and will be deleted later on.

In [27]:
cxr_folders = ensure_filepath('/content/cxr_folders')

os.makedirs(cxr_folders, exist_ok=True)

#### Temporary Folder Creation & Check

>The splitfolders library is extremely effective and very easy to use. It allows us to feed in a directory address (base_folder variable) and give it a place to put the new train, test, and validation folders (output variable). Note that the output address must already be existing. We also set a seed in order to help with model score reproducibility when training our models. The ratio parameter required floats that determines the percentage of data going to each folder.
- For more info about the splitfolders library, click here: https://github.com/jfilter/split-folders

In [28]:
splitfolders.ratio(input=cxr_base_folder, 
                    output=cxr_folders, 
                    seed=42, ratio=(0.7, 0.1, 0.2)) # ratio order: train, val, test

Copying files: 21165 files [00:03, 6283.45 files/s]


## Recommendation:

>If you prefer working with folder structures when dealing with training and testing data, I would highly recommend using the splitfolders library. Through the use of this library, I was able to create train, test, and validation folders, create image data generators, and have those generators access the data by flowing the data from my directories. 

In [31]:
folder_check(new_dir=cxr_folders, orig_dir=cxr_base_folder, check=1, )

/content/cxr_folders/train/
Number of items in COVID: 2531
Number of items in Lung_Opacity: 4208
Number of items in Viral Pneumonia: 941
Number of items in Normal: 7134
Total number of items: 14814

/content/cxr_folders/test/
Number of items in COVID: 724
Number of items in Lung_Opacity: 1203
Number of items in Viral Pneumonia: 270
Number of items in Normal: 2039
Total number of items: 4236

/content/cxr_folders/val/
Number of items in COVID: 361
Number of items in Lung_Opacity: 601
Number of items in Viral Pneumonia: 134
Number of items in Normal: 1019
Total number of items: 2115

/content/COVID-19_Radiography_Dataset/
Number of items in COVID: 3616
Number of items in Lung_Opacity: 6012
Number of items in Viral Pneumonia: 1345
Number of items in Normal: 10192
Total number of items: 21165

Are the image numbers equal?
True

Total image count: 21165


In [ ]:
# Creating variables to hold our addresses for our train, test, and val folders
train_folder, test_folder, val_folder = create_ttv(cxr_folders)

In [ ]:
train_folder

>Since we are using a DummyClassifier as our baseline model, it cannot work with ImageDataGenerators. So instead, we will create X and y train, test, and validation variables by feeding in a batch size equal to the amount of data in each train, test, and validation folder.

In [ ]:
TRAIN_BATCH_SIZE, TEST_BATCH_SIZE, VAL_BATCH_SIZE = batch_sizes([train_folder, 
                                                                 test_folder, 
                                                                 val_folder])
print(f"Train batch size: {TRAIN_BATCH_SIZE}\nTest batch size: {TEST_BATCH_SIZE}"
      f"\nValidation batch size: {VAL_BATCH_SIZE}")

#### Setting up ImageDataGenerators

>Setting up our variables for the dummy classifier

In [ ]:
# Defining our image size
IMG_SIZE = (128, 128)

# Creating ImageDataGenerators for train and test/val
train_datagen = ImageDataGenerator(rescale=1./255)
test_val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
dummy_train_generator = train_datagen.flow_from_directory(train_folder, 
                                                   target_size=IMG_SIZE,
                                                   batch_size=TRAIN_BATCH_SIZE,
                                                   class_mode='categorical')

dummy_test_generator = test_val_datagen.flow_from_directory(test_folder, 
                                                     target_size=IMG_SIZE,
                                                     batch_size=TEST_BATCH_SIZE,
                                                     class_mode='categorical')

dummy_val_generator = test_val_datagen.flow_from_directory(val_folder, 
                                                    target_size=IMG_SIZE,
                                                    batch_size=VAL_BATCH_SIZE,
                                                    class_mode='categorical')

In [ ]:
# Viewing the classes within our iterator
dummy_train_generator.class_indices

>Here is where we create our X and y variables for each train, test, and validation set. Notice that by calling next() on each iterator, we are feeding X and y variables data and labels equivalent to the batch sizes of the respective entire folder. 

In [ ]:
start = time.time()
# Creating X and y values for dummy train, test, and validation sets
X_train, y_train = next(dummy_train_generator)
X_test, y_test = next(dummy_test_generator)
X_val, y_val = next(dummy_val_generator)

end = time.time()
time_count(start, end)

In [ ]:
random_image(X_train, np.argmax(y_train, axis=1), verbose=1)

#### Baseline Model

>We need to one-hot encode our classes from our generated labels in order for the confusion matrix to be able to understand the we are working with more than two classes.

In [ ]:
# Creating and fitting our dummy classifier
dummy = DummyClassifier(strategy='stratified')
dummy.fit(X_train, y_train)

# Creating our y_pred variable
y_pred = dummy.predict(X_test)

y_pred = np.argmax(y_pred, axis=1)
y_test_ohe = np.argmax(y_test, axis=1)

In [ ]:
#Printing classification report and plotting confusion matrix
print(metrics.classification_report(y_test_ohe, y_pred));

plt.figure(figsize=(7, 6))
cm = metrics.confusion_matrix(y_test_ohe, y_pred, labels = [0, 1, 2], 
                              normalize='true')
sns.heatmap(cm, cmap="Greens", annot=True, square=True)
plt.show()

>We built a baseline dummy classifier model that has an accuracy of 34%, and it tends to guess for the class == 0, which is our Covid class. Since our dataset is relatively balanced in terms of classes, we expect to see the accuracy be around 33%.

## First CNN Model for CXRs

>Now we will create new generators that we will feed into a CNN model. Notice for the test generator we must set the parameter shuffle=False. This is because when we want to evaluate the model, we want be able to see the model's actual score against the test generator. If we were to set shuffle=True for the test generator, the labels in the test generator would not line up with the images when trying to evaluate the model.

In [ ]:
train_generator = train_datagen.flow_from_directory(train_folder, 
                                                   target_size=IMG_SIZE,
                                                   batch_size=32,
                                                   class_mode='categorical')

test_generator = test_val_datagen.flow_from_directory(test_folder, 
                                                     target_size=IMG_SIZE,
                                                     batch_size=32,
                                                     class_mode='categorical', 
                                                     shuffle=False)

val_generator = test_val_datagen.flow_from_directory(val_folder, 
                                                    target_size=IMG_SIZE,
                                                    batch_size=32,
                                                    class_mode='categorical')

In [ ]:
# Defining input shape to feed into our CNN
INPUT_SHAPE = train_generator.image_shape

#### Viewing our Class Weights

>We will now create a dictionary containing the weights for each class by using our custom make_class_weights() function.

In [ ]:
class_weights_dict = make_class_weights(train_generator)

### Creating CNN Model

In [ ]:
def create_basic_cnn(shape='',
                     metrics=['acc', tf.keras.metrics.Precision(name='precision'),
                              tf.keras.metrics.Recall(name='recall'), 
                              tf.keras.metrics.AUC(name='auc')]):
  """
  Definition:
  Creates a basic cnn model consisting of three layers and an output layer.

  Args:
  shape: this function requires an input shape in order for the model to be created.

  Returns:
  Returns a compiled model.
  """
  # We are requiring an input shape since we print the model.summary() at the 
  # end of the function
  if len(shape) < 1:
    print('Please define input shape!')
  else:
    # Input layer
    model = Sequential()
    model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=shape))
    model.add(MaxPooling2D((2, 2)))

    # Second layer
    model.add(layers.Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
      
    # Third layer
    model.add(Flatten())
    model.add(layers.Dense(128, activation='relu'))
      
    # Output layer
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=metrics)
    print(model.summary())
    return model

In [ ]:
basic_model = create_basic_cnn(shape=INPUT_SHAPE)

### Fitting model, Plotting scores, Reporting evaluation

>The fit_plot_report_gen() function takes in a CNN model and fits it on a training generator. After each epoch the validation generator is evaluated, which allows us to see if the model is learning or overfitting. Once the model finishes training, we evaluate the test generator and create a classification report, confusion matrix, and a plot for each metric that was scored over the epochs.

In [ ]:
hist1 = fit_plot_report_gen(basic_model, train_generator, test_generator, val_generator, 
                class_weights=class_weights_dict, epochs=5, batch_size=32)

>While the above model seems to be performing extremely well with an accuracy of 94.6%, we would like to improve the recall rate (94.6%) for our COVID class. Since COVID-19 is able to spread so easily, the less people that are diagnosed as false negatives, the better. 6% may seem like a small number of false negatives, but if we were to diagnose 1,000,000 people who were COVID positive, we would have around 60,000 COVID positive cases that were wrongly identified as being another class.

>Based on prior experience when working with pneumonia chest x-rays in my previous project, it is extremely important for us to feed in chest x-ray images of high quality in order to score extremely well. For this next portion, we will be using CLAHE as a preprocessing technique to enhance the images before they are fed into the model. In doing so, hopefully we will see an increase in recall rate for the COVID class and an overall increase in accuracy. 

>- Link to [Pneumonia Project repo](https://github.com/Lewis34cs/cloudy_lungs)
  - Part 1: [`pneumonia_proj.ipynb`](https://colab.research.google.com/drive/1pMKRsWUJ5pnAGUp_W01nB9fitP57F0Sc)

In [ ]:
# removing unpreprocessed CXR train, test, and val folder directory
try:
  shutil.rmtree(cxr_folders)
except:
  pass

## CLAHE Preprocessing Techniques

>Contrast Limited Adaptive Histogram Equalization (CLAHE) is used to equalize pixel intensity in images. It is very similar to Adaptive Histogram Equalization (AHE), except it doesn't over-amplify the contrast of the image. This is controlled by the clipLimit parameter. The way CLAHE works on an image is that it focuses on small portions of the image (tileGridSize parameter) and then combines these portions together through bilinear interpolation to help remove any artificial boundaries, which means that it enhances the local contrast of the total image. This essentially helps with the pixel intensity distribution, allowing us to see more "depth" in an image.

>link for info on cv2.createCLAHE(): https://docs.opencv.org/master/d6/dc7/group__imgproc__hist.html#gad689d2607b7b3889453804f414ab1018

>Here we are splitting up classes so we are able to view an example image of each class before CLAHE and then comparing the before-and-after images side by side.

In [ ]:
# Defining classes
covid = ('Covid Class', 'COVID/COVID')
normal = ('Healthy Class', 'NORMAL/NORMAL')
pneumo = ('Viral Pneumonia Class', 'Viral Pneumonia/Viral Pneumonia')

In [ ]:
class_select = input(f'Please select one of the classes to view: '
                     f'[covid][normal][pneumo] ')
if class_select == 'covid':
  selected_class = covid
elif class_select == 'normal':
  selected_class = normal
elif class_select == 'pneumo':
  selected_class = pneumo
try:
  # choosing a random image within the selected class folder
  i = np.random.choice(range(len(os.listdir(cxr_base_folder+selected_class[1].split('/')[0]))))
  img = cv2.imread(cxr_base_folder + selected_class[1] + f' ({i}).png', 0)
  print(f'Successfully loaded in an image from {selected_class[0]}')
  print(f"Image size: {img.shape}")
except:
  print('Error: No class with that name exists')

In [ ]:
# Plotting an image to view, along with distribution of pixel intensity

plt.title(selected_class[0])
plt.imshow(img, cmap='gray')
hist,bins = np.histogram(img.flatten(),256,[0,256])

cdf = hist.cumsum()
cdf_normalized = cdf * hist.max()/ cdf.max()

plt.figure(figsize=(6, 6))
plt.title('Distribution of Pixel Intensity')
plt.plot(cdf_normalized, color = 'b')
plt.hist(img.flatten(),256,[0,256], color = 'r')
plt.xlim([0,256])
plt.legend(('cdf','histogram'), loc = 'upper left')
plt.show()


>As we can see here, the distribution of pixel intensity in our unpreprocessed image ranges from 0 to around 220. The way CLAHE works on an image is that it focuses on small portions of the image (tileGridSize parameter) and then combines these portions together through **bilinear interpolation** = a resampling method that uses the distance-­weighted average of the X nearest pixel values to estimate a new pixel value. Bilinear interpolation helps remove any artificial boundaries, which enhances the local contrast of the total image.

### Creating CLAHE preprocessed images and placing them into new directory

>Creating a new folder in the 'cxr_clahe_folder' variable to store our CLAHE preprocessed images. If you would like to store the folders in another filepath, please specify where you would like to store your train, test, and validation folders in the 'cxr_clahe_folder' variable. Note that this folder is temporary and will be deleted later on.

In [ ]:
# Please define a filepath where you would like the CLAHE preprocessed images to
# be stored in the 'new_folder' variable. This folder is temporary.
try:
  if os.path.exists(cxr_clahe_folder):
    print('CLAHE folder already exists')
except:
  cxr_clahe_folder = ensure_filepath('/content/clahe_radio')
  clahe_preprocessing(cxr_base_folder, cxr_clahe_folder)

In [ ]:
num_items(cxr_clahe_folder)

In [ ]:
# Setting original and CLAHE images to compare
orig_img = cv2.imread(cxr_base_folder + selected_class[1] + f' ({i}).png')
clahe_img = cv2.imread(cxr_clahe_folder + selected_class[1] + f' ({i}).png')

In [ ]:
hist,bins = np.histogram(clahe_img.flatten(),256,[0,256])

cdf = hist.cumsum()
cdf_normalized = cdf * hist.max()/ cdf.max()

plt.figure(figsize=(6, 6))
plt.title('Distribution of Pixel Intensity Using CLAHE')
plt.plot(cdf_normalized, color = 'b')
plt.hist(clahe_img.flatten(),256,[0,256], color = 'r')
plt.xlim([0,256])
plt.legend(('cdf','histogram'), loc = 'upper left')
plt.show()


>Note that we can see that the intensities of pixels, when compared to the original distribution, have become more spread out. This will give our image a 'deeper' look to it, and give our model more information for each image.

In [ ]:
# Comparing original image (left) to CLAHE applied image (right)
res = np.hstack((orig_img,clahe_img))
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)
plt.imshow(res, cmap='gray')
plt.title('Original image                           Modified CLAHE image')
ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)
ax.set_frame_on(False)
plt.show()

>We can definitely see more of the infiltrate in the lung areas of the chest x-ray. These infiltrate areas of the lung can determine whether or not a person has Pneumonia. We can definitely see more of the infiltrate in the lung areas of the chest x-ray. These infiltrate areas of the lung can determine whether or not a person has Pneumonia. According to Hosseiny et al., when there is radiographic appearances of multifocal ground glass opacity, linear opacities, and consolidation, these are usually seen in cases of coronavirus type infections, including COVID-19, SARS, and MERS.

In [ ]:
# Removing unpreprocessed images from our directory
try:
  shutil.rmtree(cxr_base_folder)
except:
  pass

>Splitting our preprocess image folder into train, test, and validation subfolders into a new directory. If you would like to store the folders in another filepath, please specify where you would like to store your train, test, and validation folders in the 'clahe_ttv_folder' variable. Note that this folder is temporary and will be deleted later on.

In [ ]:
clahe_ttv_folder = ensure_filepath('/content/clahe_ttv')

splitfolders.ratio(input=cxr_clahe_folder, 
                    output=clahe_ttv_folder, 
                    seed=42, ratio=(0.7, 0.1, 0.2))

In [ ]:
# Removing base clahe images folder from our directory
try:
  if os.path.exists(clahe_ttv_folder):
    shutil.rmtree(cxr_clahe_folder)
except:
  print('Please run the cell above first')

In [ ]:
folder_check(clahe_ttv_folder)

## Modeling with CLAHE preprocessed images

>Defining new train, test, and validation folders for our CLAHE preprocessed chest x-ray images.

In [ ]:
train_folder, test_folder, val_folder = create_ttv(clahe_ttv_folder)

In [ ]:
# Still the same size as our previous model
IMG_SIZE = (128, 128)

In [ ]:
train_clahe_datagen = ImageDataGenerator(rescale=1./255)

test_val_clahe_datagen = ImageDataGenerator(rescale=1./255)

cl_train_generator = train_clahe_datagen.flow_from_directory(train_folder, 
                                                   target_size=IMG_SIZE,
                                                   batch_size=32,
                                                   class_mode='categorical')

cl_test_generator = test_val_clahe_datagen.flow_from_directory(test_folder, 
                                                     target_size=IMG_SIZE,
                                                     batch_size=32,
                                                     class_mode='categorical', 
                                                     shuffle=False)

cl_val_generator = test_val_clahe_datagen.flow_from_directory(val_folder, 
                                                    target_size=IMG_SIZE,
                                                    batch_size=32,
                                                    class_mode='categorical')

In [ ]:
INPUT_SHAPE = cl_train_generator.image_shape
print(INPUT_SHAPE)

In [ ]:
class_weights_dict = make_class_weights(cl_train_generator)

In [ ]:
basic_prep_cnn = create_basic_cnn(shape=INPUT_SHAPE)

>Now that we've preprocessed our images, put the images into train, test, and validation folders, and created generators to feed into our model, we can now fit the model. Feel free to tweak the parameters of the three_callbacks() function. Make sure to read how the function works first if you choose to do so. Below are some recommendations for tweaking the function.

In [ ]:
"""
Recommended parameters to play around with:
 - epochs
 - lr_patience
 - factor
 - patience
 - restore_best_weights

Please read the three_callbacks() function before changing!

Note that: 
 - patience should not be lower than lr_patience, otherwise learning rate decay
 will not occur.
 - Both patience and lr_patience should not be as large / larger than the number 
 of epochs
"""

# Please define a filepath to store all models that will be saved from the 
# ModelCheckpoint callback in the variable 'cxr_model_storage'. 
cxr_model_storage = ensure_filepath('/cxr_models/')

prep_hist = fit_plot_report_gen(basic_prep_cnn, cl_train_generator, 
                                cl_test_generator, cl_val_generator, 
                                epochs=10, batch_size=32,
                                class_weights=class_weights_dict, 
                                callbacks=three_callbacks(lr_patience=3, factor=0.9, 
                                                          patience=5, 
                                                          restore_best_weights=False,
                                                          f_path=cxr_model_storage))

#### Viewing what the model sees using lime image explainer

>By using the lime library, we create a function that explains what the model determines to be important factors (both positive and negative) when diagnosing an image. Positive factors are marked with green, and negative factors are marked with red. 

>For more information about the lime library: https://lime-ml.readthedocs.io/en/latest/lime.html#module-lime.lime_image

In [ ]:
explain_image(basic_prep_cnn, cl_train_generator, num_samples=2000, num_feats=3, 
              class_label=0)

### Saving best CXR models

>Now we define a filepath where we would like to save our best models. Please set a filepath for the 'best_models_filepath' variable where you would like to store your best models. Feel free to change the threshold, note that the lower the threshold, the more
models will be saved to the 'best_models_filepath'. Models that do not surpass the threshold are removed.

In [ ]:
best_models_filepath = ensure_filepath('/content/drive/MyDrive/gdrive/models/capstone_models/cxr_models/')
THRESHOLD = 0.96

move_best_models(source=cxr_model_storage, threshold=THRESHOLD,
                 new_dir=best_models_filepath, 
                 test_gen=cl_test_generator)

>To load in a saved model, please specify the filepath where you saved the model and put into the 'model_fpath' variable.

In [ ]:
model_fpath = '/content/drive/MyDrive/gdrive/models/capstone_models/cxr_models/model-05-0.101.hdf5'

best_model = load_model(model_fpath)
class_report_gen(best_model, cl_test_generator, 
                 class_indices=test_generator.class_indices, cmap='Greens')
best_model.evaluate(cl_test_generator, verbose=1)

## Recommendation: 
>When working with images like x-rays or MRI scans, I highly recommend using CLAHE as a preprocessing technique to create new images that give the model more to learn from. CLAHE also is able to provide enough contrast to the image without overamplifying the intensity of the pixels. It is a great tool if the goal of your project involve detection and/or recognition.

In [ ]:
# If you are finished with the above section and would like to continue, please
# set run = 1 to remove unecessary folders for Part 2
ans = input(f"All temporary files from Part 1 will be removed from the working "
            f"directory. Proceed? [y][n] ")
if ans.lower() == 'y':
  print("Removing files")
  try:
    shutil.rmtree(clahe_ttv_folder)
    shutil.rmtree(cxr_model_storage)
    print("Process completed")
  except:
    print("No folders were removed")
else:
  print("No folders were removed")

# Part 1 Conclusion
>The scores of our best model show an improvement in both recall rate (98%) for the COVID class and overall accuracy (97%). By using CLAHE, our model was able to reduce the amount of false negatives from our previous model by 4%. CLAHE is a great preprocessing tool that many hospitals and facilities should use when diagnosing their patients for certain diseases that may affect the targeted x-ray area.


#### Caveats
>While using x-rays to diagnose patients with COVID has proven to be successful with the model we've created, the process of each person getting an x-ray to determine if they have COVID can be costly. Furthermore, we wouldn't want someone with COVID to come into a facility and expose healthy people to the virus. In Part 2, we are going to explore the possiblities of using cough audio from healthy and COVID infected individuals and see if we can create a model that can accurately diagnose those with COVID. 

## Part 2: Mel-Spectrogram Binary Classification

Link to Part 2: [`covid_proj_p2_audio.ipynb`](https://colab.research.google.com/drive/1s4z-GJnNkE2NCx5MVccXxXzvBIc__tTX#scrollTo=WjwyDrj8S80W)

## References

1. Virginia Department of Health. “COVID-19 Testing.” Vdh.Virginia.Gov, Virginia Department of Health, 5 Feb. 2021, www.vdh.virginia.gov/coronavirus/covid-19-testing/#:~:text=There%20are%20two%20main%20types,)%20and%20antibody%20tests.
2. Shmerling, Robert H. “Which Test Is Best for COVID-19?” Harvard Health Blog, 5 Jan. 2021, www.health.harvard.edu/blog/which-test-is-best-for-covid-19-2020081020734#:%7E:text=The%20reported%20rate%20of%20false,infection%20the%20test%20is%20performed.
3. “Coronavirus Disease 2019 (COVID-19).” Centers for Disease Control and Prevention, 11 Feb. 2020, www.cdc.gov/coronavirus/2019-ncov/more/scientific-brief-sars-cov-2.html#:%7E:text=%E2%80%A2%20Contact%20transmission%20is%20infection,(typically%20hours).
4. Smyth, Tamara. “The Mel Scale.” University of California, San Diego, Tamara Smyth, 4 June 2019, musicweb.ucsd.edu/~trsmyth/pitch2/Mel_Scale.html.
5. M.E.H. Chowdhury, T. Rahman, A. Khandakar, R. Mazhar, M.A. Kadir, Z.B. Mahbub, K.R. Islam, M.S. Khan, A. Iqbal, N. Al-Emadi, M.B.I. Reaz, M. T. Islam, “Can AI help in screening Viral and COVID-19 pneumonia?” IEEE Access, Vol. 8, 2020, pp. 132665 - 132676.
6. Lara Orlandic, Tomas Teijeiro, & David Atienza. (2020). The COUGHVID crowdsourcing dataset: A corpus for the study of large-scale cough analysis algorithms (Version 1.0) [Data set]. Zenodo. http://doi.org/10.5281/zenodo.4048312
7. Chaudhari, Gunvant, et al. "Virufy: Global Applicability of Crowdsourced and Clinical Datasets for AI Detection of COVID-19 from Cough." arXiv preprint arXiv:2011.13320 (2020).
8. Hosseiny M, Kooraki S, Gholamrezanezhad A, Reddy S, Myers L. Radiology Perspective of Coronavirus Disease 2019 (COVID-19): Lessons From Severe Acute Respiratory Syndrome and Middle East Respiratory Syndrome. AJR Am J Roentgenol2020;214:1078-82. doi:10.2214/AJR.20.22969 pmid:32108495
9. Zafra, Miguel Fernández. “Understanding Convolutions and Pooling in Neural Networks: A Simple Explanation.” Medium, 25 May 2020, towardsdatascience.com/understanding-convolutions-and-pooling-in-neural-networks-a-simple-explanation-885a2d78f211.

# Appendix

In [ ]:
!pip install git+https://github.com/rcmalli/keras-squeezenet.git
!pip install keras_applications==1.0.4 --no-deps

In [ ]:
import tensorflow as tf

import numpy as np
import tensorflow.keras
import 
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.preprocessing import image

In [ ]:
import numpy as np
import keras.applications.
from keras_squeezenet import SqueezeNet
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.preprocessing import image